In [1]:
cd ..

/data/vision/beery/scratch/evelyn/task_datacomp


In [11]:
import json
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from visualizations.file_utils import get_per_class_json_paths

In [3]:
baselines_list = ["no_filter", "random_filter", "match_dist", "gradmatch", "zcore"]

# Instead of using a config to define the 'tasks' we want to evaluate, we define them here
dataset_list = [
    ('iWildCam', 'val1', 'test1'),
    ('iWildCam', 'val2', 'test2'),
    ('iWildCam', 'val3', 'test3'),
    ('iWildCam', 'val4', 'test4')
]

finetune_list = ["full_finetune_resnet50"]
lr_list = [0.001]
batch_size_list = [128]

In [4]:
json_paths = get_per_class_json_paths(baselines_list, dataset_list, finetune_list, lr_list, batch_size_list)

Getting pt_path for param configuration: {'fraction': 1}
Getting pt_path for param configuration: {'fraction': 0.25}
Getting pt_path for param configuration: {'fraction': 0.5}
Getting pt_path for param configuration: {'fraction': 0.75}
Getting pt_path for param configuration: {'fraction': 0.9}
Getting pt_path for param configuration: {'fraction': 0.25}
Getting pt_path for param configuration: {'fraction': 0.5}
Getting pt_path for param configuration: {'fraction': 0.75}
Getting pt_path for param configuration: {'fraction': 0.9}
Getting pt_path for param configuration: {'fraction': 0.25, 'selection_lr': 0.0001, 'selection_batch': 16}
Getting pt_path for param configuration: {'fraction': 0.5, 'selection_lr': 0.0001, 'selection_batch': 16}
Getting pt_path for param configuration: {'fraction': 0.25, 'n_sample': 500, 'redund_nn': 1000, 'sample_dim': 6}
Getting pt_path for param configuration: {'fraction': 0.25, 'n_sample': 5000, 'redund_nn': 1000, 'sample_dim': 2}


In [16]:
deployment = "test2"
deployment_json_paths = [json_path for json_path in json_paths if deployment in json_path]
deployment_json_paths

['/data/vision/beery/scratch/evelyn/task_datacomp/experiments/iWildCam/no_filter_fraction_1/test2_full_finetune_resnet50_lr=0.001_batchsize=128_logits_class_accuracy.json',
 '/data/vision/beery/scratch/evelyn/task_datacomp/experiments/iWildCam/random_filter_fraction_0.25/test2_full_finetune_resnet50_lr=0.001_batchsize=128_logits_class_accuracy.json',
 '/data/vision/beery/scratch/evelyn/task_datacomp/experiments/iWildCam/random_filter_fraction_0.5/test2_full_finetune_resnet50_lr=0.001_batchsize=128_logits_class_accuracy.json',
 '/data/vision/beery/scratch/evelyn/task_datacomp/experiments/iWildCam/random_filter_fraction_0.75/test2_full_finetune_resnet50_lr=0.001_batchsize=128_logits_class_accuracy.json',
 '/data/vision/beery/scratch/evelyn/task_datacomp/experiments/iWildCam/random_filter_fraction_0.9/test2_full_finetune_resnet50_lr=0.001_batchsize=128_logits_class_accuracy.json',
 '/data/vision/beery/scratch/evelyn/task_datacomp/experiments/iWildCam/match_dist_fraction_0.25/test2_full_fi

In [17]:
# Helper to extract a readable method name from the path
def extract_method_name(path: str) -> str:
    parts = Path(path).parts
    # typically ".../iWildCam/<method_folder>/test1_full_..."
    method_folder = parts[-2]  
    return method_folder  # e.g., "random_filter_fraction_0.25"

# Load all JSONs into a dict of dicts
acc_dicts = {}
for path in deployment_json_paths:
    with open(path, "r") as f:
        acc = json.load(f)
    method_name = extract_method_name(path)
    # convert string keys to float/int for sorting
    acc = {float(k): v for k, v in acc.items()}
    acc_dicts[method_name] = acc

# Build dataframe (outer join on all class ids)
df = pd.DataFrame(acc_dicts).sort_index()

In [18]:
df


,no_filter_fraction_1,random_filter_fraction_0.25,random_filter_fraction_0.5,random_filter_fraction_0.75,random_filter_fraction_0.9,match_dist_fraction_0.25,match_dist_fraction_0.5,match_dist_fraction_0.75,match_dist_fraction_0.9,zcore_fraction_0.25_n_sample_500_redund_nn_1000_sample_dim_6,zcore_fraction_0.25_n_sample_5000_redund_nn_1000_sample_dim_2
0.0,NaN,NaN,NaN,NaN,NaN,1.0000,1.0000,1.0000,1.0000,NaN,NaN
1.0,0.2249,NaN,0.7553,0.2447,0.0000,0.7906,0.7778,0.7521,0.6364,0.6960,0.4088
2.0,NaN,NaN,NaN,NaN,NaN,0.3047,0.7993,0.6631,0.5699,NaN,NaN
3.0,NaN,0.2812,NaN,NaN,NaN,0.2787,0.2951,0.2377,0.3033,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,0.1090,0.2447,0.1197,0.0293,NaN,0.6387
5.0,0.7388,0.5367,0.7709,0.6680,0.7599,0.2774,0.2744,0.1829,0.4329,0.6469,NaN
9.0,NaN,0.4803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0,NaN,NaN,0.5197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11.0,0.4480,NaN,NaN,0.8817,0.6416,NaN,NaN,NaN,NaN,NaN,NaN
12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1803,NaN
